In [ ]:
import pandas as pd
import seaborn as sns
from glob import glob
import matplotlib.pyplot as plt
from matplotlib.font_manager import fontManager, FontProperties

path = "/usr/share/fonts/opentype/linux-libertine/LinBiolinum_R.otf"
fontManager.addfont(path)
sns.set_theme(style="ticks", font=FontProperties(fname=path).get_name(), font_scale=1.2)

In [ ]:
data = pd.concat([pd.read_csv(data, names=[
    "Mode", "Variant", "Number of Processors", "Latency"
]) for data in glob('mutex-*.txt')], ignore_index=True)
data

In [ ]:
# plt.figure(figsize=(12, 3))
f = sns.boxplot(data=data[data["Mode"] == "All"],
    x="Number of Processors", y="Latency", hue="Variant", 
    hue_order=["Untrusted", "Replicated", "Quorum"], dodge=False)
f.set(ylabel="Latency (s)")
f.figure.savefig("mutex-all.pdf", bbox_inches='tight')

In [ ]:
f = sns.lineplot(data=data[data["Mode"] == "One"],
    x="Number of Processors", y="Latency", hue="Variant", 
    hue_order=["Untrusted", "Replicated", "Quorum"],
    style="Variant", markers=["o"])
f.set(ylim=(0, 2.0))
f.set(ylabel="Latency (s)")
f.figure.savefig("mutex-one.pdf", bbox_inches='tight')